# **TODO**


1.   Sistemare i notebook
2.   (Fabiano) Spostare tutto quello che ho fatto ieri in un singolo notebook


1.   Creazione del dataset in un unico notebook e salvare la struttura dati da leggere prima del fine tuning
2.   Voce elenco





In [25]:
!pip install -q tf-models-official
!pip install -q transformers
!pip install -q spacy
!pip install -q datasets
# !pip install -q pyarrow

ERROR: Invalid requirement: '.q'


In [62]:
import tensorflow as tf
import tensorflow_models as tfm
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# import pyarrow as pa
# import pyarrow.dataset as ds
# from datasets import Dataset
import datasets
import os
# from numba import cuda


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")
stopwords_en = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#from huggingface_hub import notebook_login

#notebook_login()

In [28]:
csv = pd.read_csv("drive/Shareddrives/BPM PROJECT/Dataset/amazon_reviews_reduced.csv", on_bad_lines="skip")
csv.dropna(inplace=True)
csv.groupby(by="product_id").count().sort_values(by="review_body")

,Unnamed: 0,marketplace,customer_id,review_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
product_id,,,,,,,,,,,,,,,
B0001FTVEK,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792
B0012S4APK,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358
B003EM8008,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766
B0002L5R78,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166
B003L1ZYYM,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333


In [29]:
csv_r = csv[csv["product_id"] == "B003L1ZYYM"]
csv_r = csv_r[["review_body", "review_headline"]]
csv_r.reset_index(inplace=True, drop=True)
review_headline = csv_r["review_headline"].tail(1000).tolist()
review_list = csv_r["review_body"].tail(1000).tolist()

In [30]:
review_list

['Work perfect to plug my laptop to my TV.<br />No more lines, or sound interference!<br /><br />Definitely I will buy it again....',
 'The clarity was amazing when we connected the cable to our computer and tv. Even if the print ont he comp was not on good it was way better on tv.',
 "What can I say? It works,<br /><br />Ordered one for my PS3. I had some problems with an expensive cable I bought years ago, the screen was flickering all the time.<br /><br />Don't spend more money, get this cable.",
 "This was the best thing I could've ever purchased! I save so much money each month because I see no need for DirecTV now. I love my apple tv! J plan on buying one more! This is a must for the apple tv and a great price!",
 'Two complaints. First off the cable is quite thick and difficult to straighten the coils out of. Secondly, the plastic plug surrounding the hdmi connector is so thick it would not plug into my TV as the plastic contacted the TVs back panel when it was less than half wa

In [19]:
#dataset = ds.dataset(pa.Table.from_pandas(csv_r).to_batches())
dataset = Dataset(pa.Table.from_pandas(csv_r[:2000]))
dataset = dataset.train_test_split(test_size=0.2)

In [20]:
p = dataset.get("train")
max_l = 0
for elem in p:
  max_l = max_l + len(elem["review_body"])
max_l/1000.0

106.4

In [21]:
prefix = "summarize: "
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["review_body"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["review_headline"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [22]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [23]:
# SPLIT DATASET
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="facebook/bart-base", return_tensors="tf")

tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=False,
    batch_size=4,
    collate_fn=data_collator,
)

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [24]:
from transformers.keras_callbacks import PushToHubCallback
model.compile(optimizer=optimizer)

model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

Epoch 1/3
400/400 [==============================] - 113s 184ms/step - loss: 1.7275 - val_loss: 1.4282
Epoch 2/3
400/400 [==============================] - 66s 165ms/step - loss: 1.3013 - val_loss: 1.4050
Epoch 3/3
400/400 [==============================] - 69s 172ms/step - loss: 1.1090 - val_loss: 1.4220


In [72]:
import re
import string
#Creation of a list of reviews, each review will be tokenized considering a word
# as a token.

print(review_list[0])

#regex to remove HTML tags
html_removal = re.compile('<.*?>')

cleaned_reviews = []
for review in review_list:
    #each review must be converted to lowercase
    lower_review = review.lower()

    #remove the html tags
    nohtml_review = re.sub(html_removal, ' ', lower_review)

    #tokenized the review using the word_tonkenize
    tokenized_review = word_tokenize(nohtml_review)

    #stopwords removal
    nostopwords_review = []

    #for each token, check if the token is inside the list of stopwords
    # for token in tokenized_review:
    #     if token not in stopwords_en:
    #         #append the ones not in the list of stopwords
    #         nostopwords_review.append(token)
    nostopwords_review = [token for token in tokenized_review if token not in stopwords_en]

    #remove punctuation #1
    #remove a token only if it is composed by a single punctuation mark
    #nopunctuation_review = [token for token in nostopwords_review if token not in string.punctuation]

    #remove punctuation #2
    #remove tokens composed entirely by punctuation marks
    nopunctuation_review = [token for token in nostopwords_review if not all(char in string.punctuation for char in token)]

    #remove punctuation #3 #we don't need to use tagging for punctuation!
    # pos_tagging = nltk.pos_tag(nostopwords_review)
    # nopunctuation_review = [tuple for tuple in pos_tagging if len(tuple[1]) > 1 ]

    cleaned_reviews.append(' '.join(nopunctuation_review))

print(cleaned_reviews[0])

Work perfect to plug my laptop to my TV.<br />No more lines, or sound interference!<br /><br />Definitely I will buy it again....
work perfect plug laptop tv lines sound interference definitely buy


In [74]:
reviews = []
for i in range(0, 400):
    sequence = cleaned_reviews[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("T5 summary nltk: " + summary)

KeyboardInterrupt: ignored

In [109]:
# reviews = []
# for i in range(0, 400):
#     sequence = cleaned_reviews[i]
#     inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
#     output = model.generate(inputs, max_length=100)
#     summary=tokenizer.decode(output[0])
#     reviews.append(summary)
# seq = ' '.join(reviews)
# inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
# output = model.generate(inputs, max_length=100)
# summary=tokenizer.decode(output[0])
# print("T5 summary nltk: " + summary)
input = "summarize: "
count = 0
checkpoint = 0


#batch creation
#creo un array di testi concatenati, ognuno dei quali ha massimo 1024 token
#lo stesso procedimento verrà fatto con i riassunti generati!
# si parte da checkpoint in poi, fino a finire il ciclo.

batches = []
i = 0
for i in range(0,100):
  if count + len(cleaned_reviews[i].split(" ")) < 1024:
    #print(cleaned_reviews[i])
    input = input + cleaned_reviews[i] + " . "
    #print(input)
    count = count + len(cleaned_reviews[i].split(" "))
  else:
    batches.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    print("Number of reviews processed: {}".format(i))
    input = "summarize: " + cleaned_reviews[i] + " . "
    count = len(cleaned_reviews[i].split(" "))

if(count != 0):
  batches.append(input)
  print('Input {} created with a length of {} tokens.'.format(len(batches), count))
  print("Number of reviews processed: {}".format(i))


Input 1 created with a length of 978 tokens.
Number of reviews processed: 43
Input 2 created with a length of 992 tokens.
Number of reviews processed: 80
Input 3 created with a length of 605 tokens.
Number of reviews processed: 99
summarize: another one products seems like ca n't beat amazon 's price recently purchased asin b007kezmx4 roku hd streaming player asin b007f9xhay sony dvpsr510h dvd player upscaling use relatively new hd tv bedroom really n't watch lot tv play video games necessarily looking highest quality cables would even know difference anyway checked around local stores none pleased cough minimum 15.99 cable connect box tv saw amazon offering 5.50 plus free shipping felt could n't go wrong price like cable seems durable well-made found easy enough hook roku dvd players problems picture quality seems great . purchased cable connecting blu-ray player receiver cable worked well high priced one big box electronics store quality excellent 's supposed sense spending gazillion

In [110]:
summaries_first_stage = []
for input in batches:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [111]:
summaries_first_stage

['great product at a great price',
 'good picture quality at a great price',
 'AmazonBasics HDMI cable works great.']

In [ ]:
#Arrivati a questo punto possiamo avere già un'idea di cosa gli utenti pensano:
#Provare un secondo stage, ma non essendoci tanti dati utilizzerà un approccio estrattivo
#Andare avanti con l'analisi dei sentimenti, basandoci sui riassunti.

#Primo approccio (quello di sopra) riassunto di riassunti, tenendone in conto il maggior numero, 10 secondi per tutto

#Secondo approccio: (quello di giù (di Matteo?) riassunto di ogni testo e concatenazione di ogni riassunto, risulta essere lento)

In [115]:
input = "summarize: "
count = 0
checkpoint = 0
batches_second_stage = []
i = 0
for i in range(0,len(summaries_first_stage)):
  if count + len(summaries_first_stage[i].split(" ")) < 1024:
    #print(cleaned_reviews[i])
    input = input + summaries_first_stage[i] + " . "
    #print(input)
    count = count + len(summaries_first_stage[i].split(" "))
  else:
    batches_second_stage.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
    print("Number of reviews processed: {}".format(i))
    input = "summarize: " + summaries_first_stage[i] + " . "
    count = len(summaries_first_stage[i].split(" "))

if(count != 0):
  batches_second_stage.append(input)
  print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
  print("Number of reviews processed: {}".format(i))

print(batches_second_stage)

Input 1 created with a length of 18 tokens.
Number of reviews processed: 2
['summarize: great product at a great price . good picture quality at a great price . AmazonBasics HDMI cable works great. . ']


In [116]:
summaries_second_stage = []
for input in batches_second_stage:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [117]:
summaries_second_stage

['good picture quality at a great price']

In [69]:
reviews = []
for i in range(0, 50):
    sequence = review_list[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("T5 summary total: " + summary)

reviews = []
for i in range(0, 50):
    sequence = cleaned_reviews[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("T5 summary nltk: " + summary)

reviews = []
for i in range(0, 50):
    sequence = fin[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("T5 summary spacy: " + summary)

# device = cuda.get_current_device()
# device.reset()

T5 summary total: </s><s>Five Stars</s><pad>


TypeError: ignored

In [ ]:
reviews = []
for i in range(0, 50):
    sequence = review_list[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("bart-base summary total: " + summary)

reviews = []
for i in range(0, 50):
    sequence = nltk_fin[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("bart-base summary nltk: " + summary)

reviews = []
for i in range(0, 50):
    sequence = fin[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("bart-base summary spacy: " + summary)

device = cuda.get_current_device()
device.reset()

In [ ]:
device = cuda.get_current_device()
device.reset()

ELIMINAZIONE STOPWORD CON NLTK E RIDUZIONE TESTO CON SPACY

In [ ]:
nltk_test = []
for elem in review_list:
    nltk_test.append(word_tokenize(elem))

nltk_un = []
for text in nltk_test:
    temp = []
    for w in text:
        if w not in stop_words:
            temp.append(w)
    nltk_un.append(' '.join(temp))

nltk_fin = []
for elem in nltk_un:
    if len(elem) <= 1000:
        nltk_fin.append(elem)

rev = []
for doc in review_list:
    words = []
    for token in nlp(doc):
        if (token.pos_ not in ['VERB', 'ADJ', 'NOUN', 'DET', 'PUNCT']):
        #if (token.pos_ == 'VERB' or token.pos_ == 'ADJ' or token.pos_ == 'NOUN' or token.pos_ == 'DET' or token.pos_ == 'PUNCT'):
            words.append(token.lemma_)
    rev.append(words)

fin = []
for doc in rev:
    fin.append(" ".join(doc))

TEST T5 CON TESTO COMPLETO, NLTK O SPACY

In [ ]:
tokenizer_t5 = AutoTokenizer.from_pretrained('t5-base')
model_t5 = TFAutoModelForSeq2SeqLM.from_pretrained('t5-base', return_dict=True)

print()

reviews = []
for i in range(0, 50):
    sequence = review_list[i]
    inputs  = tokenizer_t5.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_t5.generate(inputs, max_length=100)
    summary=tokenizer_t5.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_t5.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_t5.generate(inputs, max_length=100)
summary=tokenizer_t5.decode(output[0])
print("T5 summary total: " + summary)

reviews = []
for i in range(0, 50):
    sequence = nltk_fin[i]
    inputs  = tokenizer_t5.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_t5.generate(inputs, max_length=100)
    summary=tokenizer_t5.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_t5.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_t5.generate(inputs, max_length=100)
summary=tokenizer_t5.decode(output[0])
print("T5 summary nltk: " + summary)

reviews = []
for i in range(0, 50):
    sequence = fin[i]
    inputs  = tokenizer_t5.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_t5.generate(inputs, max_length=100)
    summary=tokenizer_t5.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_t5.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_t5.generate(inputs, max_length=100)
summary=tokenizer_t5.decode(output[0])
print("T5 summary spacy: " + summary)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.



T5 summary total: <pad> i bought a tv cable for my pc and a tv set for $100. the cable is not as thick as the store brand ones.</s>
T5 summary nltk: <pad> i bought a new tv set for my ps3 and it works great. i'm not a fan of vga cables, but they are great value for money.</s>
T5 summary spacy: <pad> if it was a 270 degree adapter it would be a definite no-brainer. if it was a 270 degree adapter it would be a definite no-brainer.</s>


TEST BART CON TESTO COMPLETO, NLTK O SPACY

In [ ]:
tokenizer_bart = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
model_bart = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn', return_dict=True)

print()

reviews = []
for i in range(0, 50):
    sequence = review_list[i]
    inputs  = tokenizer_bart.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_bart.generate(inputs, max_length=100)
    summary=tokenizer_bart.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_bart.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_bart.generate(inputs, max_length=100)
summary=tokenizer_bart.decode(output[0])
print("Bart summary total: " + summary)

reviews = []
for i in range(0, 50):
    sequence = nltk_fin[i]
    inputs  = tokenizer_bart.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_bart.generate(inputs, max_length=100)
    summary=tokenizer_bart.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_bart.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_bart.generate(inputs, max_length=100)
summary=tokenizer_bart.decode(output[0])
print("Bart summary nltk: " + summary)

reviews = []
for i in range(0, 50):
    sequence = fin[i]
    inputs  = tokenizer_bart.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_bart.generate(inputs, max_length=100)
    summary=tokenizer_bart.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_bart.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_bart.generate(inputs, max_length=100)
summary=tokenizer_bart.decode(output[0])
print("Bart summary spacy: " + summary)

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-cnn.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.



Bart summary total: </s><s>The cable is a great way to connect your TV to your computer or phone. The cable is very easy to use. It's a great value for money. I would definitely buy one of these again. I'm glad I bought one. It was a great purchase.</s><pad>


KeyboardInterrupt: ignored

TEST PEGASUS CON TESTO COMPLETO, NLTK O SPACY

In [ ]:
tokenizer_pega = AutoTokenizer.from_pretrained('google/pegasus-xsum')
model_pega = TFAutoModelForSeq2SeqLM.from_pretrained('google/pegasus-xsum', return_dict=True)

print()

reviews = []
for i in range(0, 50):
    sequence = review_list[i]
    inputs  = tokenizer_pega.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=512)
    output = model_pega.generate(inputs, max_length=100)
    summary=tokenizer_pega.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_pega.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=512)
output = model_pega.generate(inputs, max_length=100)
summary=tokenizer_pega.decode(output[0])
print("Pegasus summary totale: " + summary)

reviews = []
for i in range(0, 50):
    sequence = nltk_fin[i]
    inputs  = tokenizer_pega.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=512)
    output = model_pega.generate(inputs, max_length=100)
    summary=tokenizer_pega.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_pega.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=512)
output = model_pega.generate(inputs, max_length=100)
summary=tokenizer_pega.decode(output[0])
print("Pegasus summary nltk: " + summary)

reviews = []
for i in range(0, 50):
    sequence = fin[i]
    inputs  = tokenizer_pega.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=512)
    output = model_pega.generate(inputs, max_length=100)
    summary=tokenizer_pega.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_pega.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=512)
output = model_pega.generate(inputs, max_length=100)
summary=tokenizer_pega.decode(output[0])
print("Pegasus summary spacy: " + summary)